<a href="https://colab.research.google.com/github/aialharbi/ACWE/blob/master/AI_Saudi_Standards_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Saudi Standards

In [ ]:
!pip install langchain --upgrade
!pip install tiktoken
!pip install pypdf
!pip install pinecone-client
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
import pinecone

In [ ]:
loader = PyPDFLoader("SBC 801.pdf") # input standards pdf file
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [ ]:
# Check your environment variable with you API keys...
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', OPENAI_API_KEY) # Use your OPENAI API KEY

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', PINECONE_API_KEY) # Use your  PINECONE API KEY
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', PINECONE_API_ENV) # Use your  PINECONE environment variable
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "NAME" # put in the name of your pinecone index here

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = "هل يجب توفير أنظمة المواسير لخشبة المسرح مساحتها 93 متر مربع"

docs = docsearch.similarity_search(query)
chain.run(input_documents=docs[:1], question=query)

' لا، لا يجب توفير أنظمة المواسير لخشبة المسرح مساحتها 93 متر مربع.'